In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pathlib
import cv2
from PIL import Image
# df=pd.read_csv(r"diabetic_data.csv")
df=pd.read_csv (r"data\readmission.csv")

In [2]:
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,17640534,13603752,Caucasian,Female,[60-70),?,3,1,1,1,...,No,No,No,No,No,No,No,Ch,Yes,NO
2746,17664828,5374836,Caucasian,Male,[40-50),?,2,1,1,1,...,No,No,No,No,No,No,No,No,No,NO
2747,17667876,340488,AfricanAmerican,Female,[70-80),?,1,1,7,5,...,No,Steady,No,No,No,No,No,No,Yes,>30
2748,17672388,3780207,Caucasian,Male,[90-100),?,1,1,7,7,...,No,No,No,No,No,No,No,No,No,NO


In [3]:
len(df['readmitted'])

2750

In [4]:
df['readmitted']=df['readmitted'].replace(['NO','>30','<30'],['0','0','1'])
df['readmitted']=df['readmitted'].astype('int')

In [5]:
#dropping encounter_id,patient_nbr as it is just identifier 
#weight as it contains 97% null values
#examide, citogliption as they have only one categorical value
#diag_1,diag_2,diag_3 as we are considering number_of_diagnosis
df=df.drop(['encounter_id','patient_nbr','weight','examide','citoglipton','diag_1','diag_2','diag_3'],axis=1)

In [6]:
df

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,?,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,[10-20),1,1,7,3,?,?,59,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,AfricanAmerican,Female,[20-30),1,1,7,2,?,?,11,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,?,?,44,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,?,?,51,...,No,Steady,No,No,No,No,No,Ch,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,Caucasian,Female,[60-70),3,1,1,1,?,Surgery-Cardiovascular/Thoracic,49,...,No,No,No,No,No,No,No,Ch,Yes,0
2746,Caucasian,Male,[40-50),2,1,1,1,?,Cardiology,42,...,No,No,No,No,No,No,No,No,No,0
2747,AfricanAmerican,Female,[70-80),1,1,7,5,?,Emergency/Trauma,56,...,No,Steady,No,No,No,No,No,No,Yes,0
2748,Caucasian,Male,[90-100),1,1,7,7,?,Surgery-General,46,...,No,No,No,No,No,No,No,No,No,0


In [7]:
df.replace('?',np.nan,inplace=True)

In [8]:
df.isnull().sum()

race                          62
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
payer_code                  2750
medical_specialty            702
num_lab_procedures             0
num_procedures                 0
num_medications                0
number_outpatient              0
number_emergency               0
number_inpatient               0
number_diagnoses               0
max_glu_serum               2657
A1Cresult                   2152
metformin                      0
repaglinide                    0
nateglinide                    0
chlorpropamide                 0
glimepiride                    0
acetohexamide                  0
glipizide                      0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitazone                  0
acarbose                       0
miglitol  

In [9]:
#discharge_disposition_id informs where the patient has gone after discharging. Ids 11,13,14,19,20,21 represent death or Hospice they cannot be returned
df=df.loc[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]

In [10]:
#Numerical Columns in dataset
df_num_cols=df[['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses']]
df_num_cols.isnull().sum()

time_in_hospital      0
num_lab_procedures    0
num_procedures        0
num_medications       0
number_outpatient     0
number_emergency      0
number_inpatient      0
number_diagnoses      0
dtype: int64

In [11]:
# dir = pathlib.Path('data')

# # Get a list of all the files in the data directory and its subdirectories
# image_files = list(dir.glob('*/*'))
# severity_labels = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

# diabetic = []

# for j in severity_labels:
#         folderPath = os.path.join(dir,j)
#         for k in tqdm(os.listdir(folderPath)):
#             img = os.path.join(folderPath,k)
#             diabetic.append(img)
# print(diabetic)

In [12]:
# df['diabetic_eye'] = diabetic


In [13]:
df

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,NaN,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,[10-20),1,1,7,3,NaN,NaN,59,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,AfricanAmerican,Female,[20-30),1,1,7,2,NaN,NaN,11,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,NaN,NaN,44,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,NaN,NaN,51,...,No,Steady,No,No,No,No,No,Ch,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,Caucasian,Female,[60-70),3,1,1,1,NaN,Surgery-Cardiovascular/Thoracic,49,...,No,No,No,No,No,No,No,Ch,Yes,0
2746,Caucasian,Male,[40-50),2,1,1,1,NaN,Cardiology,42,...,No,No,No,No,No,No,No,No,No,0
2747,AfricanAmerican,Female,[70-80),1,1,7,5,NaN,Emergency/Trauma,56,...,No,Steady,No,No,No,No,No,No,Yes,0
2748,Caucasian,Male,[90-100),1,1,7,7,NaN,Surgery-General,46,...,No,No,No,No,No,No,No,No,No,0


In [14]:
df_num_cols=pd.concat([df_num_cols],axis=1)

In [15]:
df_num_cols.columns

Index(['time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses'],
      dtype='object')

In [16]:
df_num_cols.corr()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
time_in_hospital,1.000000,0.357335,0.235716,0.523439,0.001789,-0.028370,0.055878,0.287445
num_lab_procedures,0.357335,1.000000,0.049202,0.319299,0.010656,-0.006899,-0.031072,0.222667
num_procedures,0.235716,0.049202,1.000000,0.476565,0.019773,-0.038412,-0.081921,0.193524
num_medications,0.523439,0.319299,0.476565,1.000000,0.020754,-0.007146,0.033837,0.346996
number_outpatient,0.001789,0.010656,0.019773,0.020754,1.000000,0.156280,0.143458,0.036193
number_emergency,-0.028370,-0.006899,-0.038412,-0.007146,0.156280,1.000000,0.090424,0.040568
number_inpatient,0.055878,-0.031072,-0.081921,0.033837,0.143458,0.090424,1.000000,0.117350
number_diagnoses,0.287445,0.222667,0.193524,0.346996,0.036193,0.040568,0.117350,1.000000


In [17]:
#Categorical Columns in dataset
categorical_columns=['race', 'gender','payer_code','max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed']

In [18]:
df['race']=df['race'].fillna('UKN')
df['medical_specialty']=df['medical_specialty'].fillna('UKN')
df['payer_code']=df['payer_code'].fillna('UKN')


C:\Users\roopadharshini\AppData\Local\Temp\ipykernel_12868\1826956655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race']=df['race'].fillna('UKN')
C:\Users\roopadharshini\AppData\Local\Temp\ipykernel_12868\1826956655.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['medical_specialty']=df['medical_specialty'].fillna('UKN')
C:\Users\roopadharshini\AppData\Local\Temp\ipykernel_12868\1826956655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [19]:
df

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,UKN,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,[10-20),1,1,7,3,UKN,UKN,59,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,AfricanAmerican,Female,[20-30),1,1,7,2,UKN,UKN,11,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,UKN,UKN,44,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,UKN,UKN,51,...,No,Steady,No,No,No,No,No,Ch,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,Caucasian,Female,[60-70),3,1,1,1,UKN,Surgery-Cardiovascular/Thoracic,49,...,No,No,No,No,No,No,No,Ch,Yes,0
2746,Caucasian,Male,[40-50),2,1,1,1,UKN,Cardiology,42,...,No,No,No,No,No,No,No,No,No,0
2747,AfricanAmerican,Female,[70-80),1,1,7,5,UKN,Emergency/Trauma,56,...,No,Steady,No,No,No,No,No,No,Yes,0
2748,Caucasian,Male,[90-100),1,1,7,7,UKN,Surgery-General,46,...,No,No,No,No,No,No,No,No,No,0


In [20]:
df

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,UKN,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,[10-20),1,1,7,3,UKN,UKN,59,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,AfricanAmerican,Female,[20-30),1,1,7,2,UKN,UKN,11,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,UKN,UKN,44,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,UKN,UKN,51,...,No,Steady,No,No,No,No,No,Ch,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,Caucasian,Female,[60-70),3,1,1,1,UKN,Surgery-Cardiovascular/Thoracic,49,...,No,No,No,No,No,No,No,Ch,Yes,0
2746,Caucasian,Male,[40-50),2,1,1,1,UKN,Cardiology,42,...,No,No,No,No,No,No,No,No,No,0
2747,AfricanAmerican,Female,[70-80),1,1,7,5,UKN,Emergency/Trauma,56,...,No,Steady,No,No,No,No,No,No,Yes,0
2748,Caucasian,Male,[90-100),1,1,7,7,UKN,Surgery-General,46,...,No,No,No,No,No,No,No,No,No,0


In [21]:
df1=df.groupby('medical_specialty').size().sort_values(ascending=False)

In [22]:
df1.head(10)

medical_specialty
UKN                                686
InternalMedicine                   641
Cardiology                         346
Family/GeneralPractice             309
Surgery-General                    108
Orthopedics-Reconstructive          79
Psychiatry                          73
Nephrology                          71
Surgery-Cardiovascular/Thoracic     54
ObstetricsandGynecology             51
dtype: int64

In [23]:
top_10=['UKN','InternalMedicine','Emergency/Trauma','Family/GeneralPractice','Cardiology','Surgery-General','Nephrology','Orthopedics','Orthopedics-Reconstructive','Radiologist']

In [24]:
df.loc[~df.medical_specialty.isin(top_10),'medical_specialty']= 'Other'

In [25]:
df.describe()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
count,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000,2710.000000
mean,2.979336,6.708118,5.716605,4.624354,49.573432,1.617712,14.707380,0.054613,0.014760,0.370480,6.731365,0.105535
std,2.145746,9.570514,4.529480,3.036604,16.517370,1.829171,7.556187,0.516225,0.145575,0.818312,2.109273,0.307298
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,1.000000,1.000000,1.000000,2.000000,39.000000,0.000000,9.000000,0.000000,0.000000,0.000000,5.000000,0.000000
50%,2.000000,1.000000,7.000000,4.000000,49.000000,1.000000,14.000000,0.000000,0.000000,0.000000,7.000000,0.000000
75%,6.000000,6.000000,7.000000,6.000000,61.000000,3.000000,19.000000,0.000000,0.000000,0.000000,9.000000,0.000000
max,6.000000,25.000000,20.000000,14.000000,105.000000,6.000000,61.000000,12.000000,4.000000,7.000000,9.000000,1.000000


In [26]:
df['admission_type_id']=df['admission_type_id'].astype('str')
df['discharge_disposition_id']=df['discharge_disposition_id'].astype('str')
df['admission_source_id']=df['admission_source_id'].astype('str')

C:\Users\roopadharshini\AppData\Local\Temp\ipykernel_12868\2835709561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['admission_type_id']=df['admission_type_id'].astype('str')
C:\Users\roopadharshini\AppData\Local\Temp\ipykernel_12868\2835709561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['discharge_disposition_id']=df['discharge_disposition_id'].astype('str')
C:\Users\roopadharshini\AppData\Local\Temp\ipykernel_12868\2835709561.py:3: SettingWithCopyWarning: 
A value is trying to be set o

In [27]:
df['admission_type_id'].unique()

array(['6', '1', '2', '3', '4'], dtype=object)

In [28]:
df.to_csv('df_naive.csv')

In [29]:
df_cat=pd.get_dummies(df[categorical_columns+ ['admission_type_id','discharge_disposition_id','admission_source_id','medical_specialty']],drop_first=True)

In [30]:
list(df_cat)

['race_Asian',
 'race_Caucasian',
 'race_Hispanic',
 'race_Other',
 'race_UKN',
 'gender_Male',
 'max_glu_serum_>300',
 'max_glu_serum_Norm',
 'A1Cresult_>8',
 'A1Cresult_Norm',
 'metformin_No',
 'metformin_Steady',
 'metformin_Up',
 'repaglinide_No',
 'repaglinide_Steady',
 'repaglinide_Up',
 'chlorpropamide_Steady',
 'glimepiride_No',
 'glimepiride_Steady',
 'glimepiride_Up',
 'glipizide_No',
 'glipizide_Steady',
 'glipizide_Up',
 'glyburide_No',
 'glyburide_Steady',
 'glyburide_Up',
 'tolbutamide_Steady',
 'pioglitazone_No',
 'pioglitazone_Steady',
 'pioglitazone_Up',
 'rosiglitazone_No',
 'rosiglitazone_Steady',
 'rosiglitazone_Up',
 'acarbose_Steady',
 'acarbose_Up',
 'troglitazone_Steady',
 'tolazamide_Steady',
 'insulin_No',
 'insulin_Steady',
 'insulin_Up',
 'change_No',
 'diabetesMed_Yes',
 'admission_type_id_2',
 'admission_type_id_3',
 'admission_type_id_4',
 'admission_type_id_6',
 'discharge_disposition_id_10',
 'discharge_disposition_id_2',
 'discharge_disposition_id_25',

In [31]:
df_cat

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>8,A1Cresult_Norm,...,admission_source_id_7,medical_specialty_Emergency/Trauma,medical_specialty_Family/GeneralPractice,medical_specialty_InternalMedicine,medical_specialty_Nephrology,medical_specialty_Orthopedics,medical_specialty_Orthopedics-Reconstructive,medical_specialty_Other,medical_specialty_Surgery-General,medical_specialty_UKN
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,True,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
3,False,True,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
4,False,True,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2746,False,True,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2747,False,False,False,False,False,False,False,False,True,False,...,True,True,False,False,False,False,False,False,False,False
2748,False,True,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False


In [32]:
df_num_cols

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,1,41,0,1,0,0,0,1
1,3,59,0,18,0,0,0,9
2,2,11,5,13,2,0,1,6
3,2,44,1,16,0,0,0,7
4,1,51,0,8,0,0,0,5
...,...,...,...,...,...,...,...,...
2745,1,49,2,16,0,0,0,8
2746,1,42,6,8,0,0,0,5
2747,5,56,0,11,0,0,0,5
2748,7,46,1,11,0,0,0,9


In [33]:
df['age']

0         [0-10)
1        [10-20)
2        [20-30)
3        [30-40)
4        [40-50)
          ...   
2745     [60-70)
2746     [40-50)
2747     [70-80)
2748    [90-100)
2749     [50-60)
Name: age, Length: 2710, dtype: object

In [34]:
dff=pd.concat([df_cat,df_num_cols,df['age']],axis=1)

In [35]:
len(dff)

2710

In [36]:
dff

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>8,A1Cresult_Norm,...,medical_specialty_UKN,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age
0,False,True,False,False,False,False,False,False,False,False,...,False,1,41,0,1,0,0,0,1,[0-10)
1,False,True,False,False,False,False,False,False,False,False,...,True,3,59,0,18,0,0,0,9,[10-20)
2,False,False,False,False,False,False,False,False,False,False,...,True,2,11,5,13,2,0,1,6,[20-30)
3,False,True,False,False,False,True,False,False,False,False,...,True,2,44,1,16,0,0,0,7,[30-40)
4,False,True,False,False,False,True,False,False,False,False,...,True,1,51,0,8,0,0,0,5,[40-50)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,False,True,False,False,False,False,False,False,False,False,...,False,1,49,2,16,0,0,0,8,[60-70)
2746,False,True,False,False,False,True,False,False,False,False,...,False,1,42,6,8,0,0,0,5,[40-50)
2747,False,False,False,False,False,False,False,False,True,False,...,False,5,56,0,11,0,0,0,5,[70-80)
2748,False,True,False,False,False,True,False,False,False,False,...,False,7,46,1,11,0,0,0,9,[90-100)


In [37]:
dff.dtypes

race_Asian             bool
race_Caucasian         bool
race_Hispanic          bool
race_Other             bool
race_UKN               bool
                      ...  
number_outpatient     int64
number_emergency      int64
number_inpatient      int64
number_diagnoses      int64
age                  object
Length: 80, dtype: object

In [38]:
age_id = {'[0-10)':5, 
          '[10-20)':15, 
          '[20-30)':25, 
          '[30-40)':35, 
          '[40-50)':45, 
          '[50-60)':55,
          '[60-70)':65, 
          '[70-80)':75, 
          '[80-90)':85, 
          '[90-100)':95}
dff['age_group'] = dff.age.replace(age_id)

In [39]:
dff.columns

Index(['race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other',
       'race_UKN', 'gender_Male', 'max_glu_serum_>300', 'max_glu_serum_Norm',
       'A1Cresult_>8', 'A1Cresult_Norm', 'metformin_No', 'metformin_Steady',
       'metformin_Up', 'repaglinide_No', 'repaglinide_Steady',
       'repaglinide_Up', 'chlorpropamide_Steady', 'glimepiride_No',
       'glimepiride_Steady', 'glimepiride_Up', 'glipizide_No',
       'glipizide_Steady', 'glipizide_Up', 'glyburide_No', 'glyburide_Steady',
       'glyburide_Up', 'tolbutamide_Steady', 'pioglitazone_No',
       'pioglitazone_Steady', 'pioglitazone_Up', 'rosiglitazone_No',
       'rosiglitazone_Steady', 'rosiglitazone_Up', 'acarbose_Steady',
       'acarbose_Up', 'troglitazone_Steady', 'tolazamide_Steady', 'insulin_No',
       'insulin_Steady', 'insulin_Up', 'change_No', 'diabetesMed_Yes',
       'admission_type_id_2', 'admission_type_id_3', 'admission_type_id_4',
       'admission_type_id_6', 'discharge_disposition_id_10',
       'dis

In [40]:
dff=pd.concat([dff,df['readmitted']],axis=1)

In [41]:
dff.columns

Index(['race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other',
       'race_UKN', 'gender_Male', 'max_glu_serum_>300', 'max_glu_serum_Norm',
       'A1Cresult_>8', 'A1Cresult_Norm', 'metformin_No', 'metformin_Steady',
       'metformin_Up', 'repaglinide_No', 'repaglinide_Steady',
       'repaglinide_Up', 'chlorpropamide_Steady', 'glimepiride_No',
       'glimepiride_Steady', 'glimepiride_Up', 'glipizide_No',
       'glipizide_Steady', 'glipizide_Up', 'glyburide_No', 'glyburide_Steady',
       'glyburide_Up', 'tolbutamide_Steady', 'pioglitazone_No',
       'pioglitazone_Steady', 'pioglitazone_Up', 'rosiglitazone_No',
       'rosiglitazone_Steady', 'rosiglitazone_Up', 'acarbose_Steady',
       'acarbose_Up', 'troglitazone_Steady', 'tolazamide_Steady', 'insulin_No',
       'insulin_Steady', 'insulin_Up', 'change_No', 'diabetesMed_Yes',
       'admission_type_id_2', 'admission_type_id_3', 'admission_type_id_4',
       'admission_type_id_6', 'discharge_disposition_id_10',
       'dis

In [42]:
dff.dtypes

race_Asian            bool
race_Caucasian        bool
race_Hispanic         bool
race_Other            bool
race_UKN              bool
                     ...  
number_inpatient     int64
number_diagnoses     int64
age                 object
age_group            int64
readmitted           int32
Length: 82, dtype: object

In [43]:
dff=dff.drop(['age'],axis=1)

In [44]:
dff.dtypes

race_Asian           bool
race_Caucasian       bool
race_Hispanic        bool
race_Other           bool
race_UKN             bool
                    ...  
number_emergency    int64
number_inpatient    int64
number_diagnoses    int64
age_group           int64
readmitted          int32
Length: 81, dtype: object

In [45]:
df=dff

In [46]:
df = df.sample(n = len(df))
df = df.reset_index(drop = True)

In [47]:
df_valid_test=df.sample(frac=0.30)

In [48]:
df_test = df_valid_test.sample(frac = 0.5)
df_valid = df_valid_test.drop(df_test.index)
df_train=df.drop(df_valid_test.index)

In [72]:
df

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>8,A1Cresult_Norm,...,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group,readmitted
0,False,True,False,False,False,True,False,False,False,False,...,6,12,3,12,0,0,0,4,55,0
1,False,True,False,False,False,True,False,False,False,False,...,1,34,6,9,0,0,0,6,75,0
2,False,True,False,False,False,True,False,False,False,False,...,1,41,5,10,0,0,0,7,55,0
3,False,False,False,False,False,True,False,False,False,False,...,5,49,0,12,0,0,0,9,75,0
4,False,False,False,False,False,False,False,False,False,False,...,3,35,4,13,0,0,0,8,75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2705,False,True,False,False,False,False,False,False,False,False,...,6,27,0,16,0,0,0,8,75,0
2706,False,False,False,False,False,True,False,False,True,False,...,3,77,0,12,0,0,0,4,45,0
2707,False,True,False,False,False,False,False,False,False,False,...,5,41,2,15,0,0,0,8,65,1
2708,False,False,False,False,False,True,False,False,False,False,...,2,45,0,13,0,0,0,9,55,1


In [86]:
df.to_csv('df_naive.csv',index=False)

In [49]:
def calc_prevalence(y_actual):
    return float(sum(y_actual)/len(y_actual))

In [50]:
print('Test prevalence(n = %d):%.3f'%(len(df_test),calc_prevalence(df_test.readmitted.values)))
print('Valid prevalence(n = %d):%.3f'%(len(df_valid),calc_prevalence(df_valid.readmitted.values)))
print('Train all prevalence(n = %d):%.3f'%(len(df_train), calc_prevalence(df_train.readmitted.values)))


Test prevalence(n = 406):0.133
Valid prevalence(n = 407):0.091
Train all prevalence(n = 1897):0.103


In [51]:
rows_pos = df_train.readmitted == 1
df_train_pos = df_train.loc[rows_pos]
df_train_neg = df_train.loc[~rows_pos]

# merge the balanced data
df_train = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos))],axis = 0)

# shuffle the order of training samples 
df_train = df_train.sample(n = len(df_train)).reset_index(drop = True)


In [52]:
df_train.shape

(390, 81)

In [53]:
X_train = df_train.loc[:, df_train.columns != 'readmitted'].values
y_train=df_train['readmitted'].values
X_valid =  df_valid.loc[:, df_valid.columns != 'readmitted'].values
y_valid=df_valid['readmitted'].values

In [54]:
X_test =  df_test.loc[:, df_test.columns != 'readmitted'].values
y_test=df_test['readmitted'].values

In [55]:
from sklearn.preprocessing import StandardScaler

scaler  = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [56]:
df_train.describe()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group,readmitted
count,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000
mean,4.787179,49.451282,1.441026,15.051282,0.053846,0.020513,0.553846,6.782051,61.153846,0.500000
std,2.981233,16.245749,1.740301,6.981241,0.411497,0.174432,1.017039,2.054823,16.649322,0.500642
min,1.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000
25%,3.000000,38.000000,0.000000,10.000000,0.000000,0.000000,0.000000,5.000000,47.500000,0.000000
50%,4.000000,50.000000,1.000000,15.000000,0.000000,0.000000,0.000000,7.000000,65.000000,0.500000
75%,6.000000,61.000000,2.000000,19.000000,0.000000,0.000000,1.000000,9.000000,75.000000,1.000000
max,14.000000,105.000000,6.000000,46.000000,5.000000,2.000000,7.000000,9.000000,95.000000,1.000000


In [57]:
X_train_tf = scaler.transform(X_train)
X_valid_tf = scaler.transform(X_valid)

In [58]:
X_test_tf = scaler.transform(X_test)

In [59]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [60]:
pd.DataFrame(X_train_tf)[9].unique()


array([-0.18569534,  5.38516481])

In [61]:
X_train.shape

(390, 80)

In [62]:
X_train_tf.shape

(390, 80)

In [63]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train_tf, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [85]:
import pickle
model_filename = 'Diabetic_Readmission_model.pkl'

# Save the model to a PKL file
with open(model_filename, 'wb') as model_file:
    pickle.dump(classifier,model_file)

In [64]:
y_pred=classifier.predict(X_test_tf)

In [65]:
print(y_pred)

[1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0
 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1
 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 1
 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 1 0
 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1
 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1
 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 1
 0 0 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 1 0 0 0 0 0 1 0 1 1 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 1 1
 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1]


In [66]:

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 1]
 [1 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [1 0]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [1 0]
 [0 0]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 0]
 [0 0]
 [0 1]
 [0 1]
 [1 0]
 [0 0]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [1 1]
 [0 0]
 [0 1]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [1 0]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [0 0]
 [0 1]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]

In [67]:

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[216 136]
 [ 24  30]]


0.6059113300492611

In [79]:
# test = [race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>8,A1Cresult_Norm,metformin_No,metformin_Steady,metformin_Up,repaglinide_No,repaglinide_Steady,repaglinide_Up,chlorpropamide_Steady,glimepiride_No,glimepiride_Steady,glimepiride_Up,glipizide_No,glipizide_Steady,glipizide_Up,glyburide_No,glyburide_Steady,glyburide_Up,tolbutamide_Steady,pioglitazone_No,pioglitazone_Steady,pioglitazone_Up,rosiglitazone_No,rosiglitazone_Steady,rosiglitazone_Up,acarbose_Steady,acarbose_Up,troglitazone_Steady,tolazamide_Steady,insulin_No,insulin_Steady,insulin_Up,change_No,diabetesMed_Yes,admission_type_id_2,admission_type_id_3,admission_type_id_4,admission_type_id_6,discharge_disposition_id_10,discharge_disposition_id_2,discharge_disposition_id_25,discharge_disposition_id_3,discharge_disposition_id_4,discharge_disposition_id_5,discharge_disposition_id_6,discharge_disposition_id_7,admission_source_id_17,admission_source_id_2,admission_source_id_20,admission_source_id_3,admission_source_id_4,admission_source_id_5,admission_source_id_6,admission_source_id_7,medical_specialty_Emergency/Trauma,medical_specialty_Family/GeneralPractice,medical_specialty_InternalMedicine,medical_specialty_Nephrology,medical_specialty_Orthopedics,medical_specialty_Orthopedics-Reconstructive,medical_specialty_Other,medical_specialty_Surgery-General,medical_specialty_UKN,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group]
test = np.array( [False,True,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,6,12,3,12,0,0,0,4,55] )
# scaler  = StandardScaler()

scaler.fit(test.reshape(1,-1))

StandardScaler()

In [81]:
test_tf = scaler.transform(test.reshape(1,-1))

In [82]:
prediction_test =classifier.predict(test_tf)

In [84]:
if prediction_test == 0:
    print("No need of Re-admission!!")
else:
    print("Readmission needed!!")

No need of Re-admission!!
